# Preparing the dataset
Objective: Try a second approach for the images that didn't work. 

In [308]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import os 


def save_image(name, img):
    cv.imwrite('..\datasets\ISIC2017\Training_data_processed\\not_successfully_cutted\\{}.jpg'.format(name), img)

def save_in_test(name, img):
    cv.imwrite('..\datasets\ISIC2017\Training_data_processed\\tests\\{}.jpg'.format(name), img)

def read_image(name):
    img = cv.imread('..\datasets\ISIC2017\Training_data_processed\\not_successfully_cutted\\{}.jpg'.format(name))
    return img



Obtaining all images name

In [309]:
all_files = os.listdir('..\datasets\ISIC2017\Training_data_processed\\not_successfully_cutted\\')
all_images = []
for file in all_files:
        if file.find('superpixels') == -1 and file.find('Training_Data_metadata') == -1 and file.find('_final') == -1 and file.find('cut_coords') == -1:
                new_img = file.split('.')[0]
                all_images.append(new_img)

all_images


['ISIC_0000021',
 'ISIC_0000026',
 'ISIC_0000031',
 'ISIC_0000032',
 'ISIC_0000036',
 'ISIC_0000071',
 'ISIC_0000077',
 'ISIC_0000091',
 'ISIC_0000092',
 'ISIC_0000094',
 'ISIC_0000095',
 'ISIC_0000105',
 'ISIC_0000113',
 'ISIC_0000118',
 'ISIC_0000127',
 'ISIC_0000132',
 'ISIC_0000160',
 'ISIC_0000268',
 'ISIC_0000277',
 'ISIC_0000281',
 'ISIC_0000282',
 'ISIC_0000311',
 'ISIC_0000379',
 'ISIC_0000385',
 'ISIC_0000387',
 'ISIC_0000388',
 'ISIC_0000390',
 'ISIC_0000392',
 'ISIC_0000403',
 'ISIC_0000421',
 'ISIC_0000473',
 'ISIC_0000882',
 'ISIC_0001103',
 'ISIC_0001105',
 'ISIC_0001119',
 'ISIC_0001126',
 'ISIC_0001128',
 'ISIC_0001133',
 'ISIC_0001134',
 'ISIC_0001960',
 'ISIC_0002093',
 'ISIC_0002107',
 'ISIC_0002206',
 'ISIC_0002246',
 'ISIC_0002251',
 'ISIC_0002287',
 'ISIC_0002353',
 'ISIC_0002374',
 'ISIC_0002438',
 'ISIC_0002453',
 'ISIC_0002459',
 'ISIC_0002469',
 'ISIC_0002476',
 'ISIC_0002489',
 'ISIC_0002616',
 'ISIC_0002647',
 'ISIC_0002673',
 'ISIC_0002829',
 'ISIC_0002836

Use morphological operations to better binarize the image

In [310]:
def morphological_operations(img):
    kernel = np.ones((10,10),np.uint8)
    img = cv.dilate(img,kernel,iterations = 2) 
    img = cv.erode(img,kernel,iterations = 2) 
    return img

In [311]:
# img_original = read_image('ISIC_0014395')
# img = morphological_operations(img_original)
# save_in_test('ISIC_0014395', img_original)
# save_in_test('ISIC_0014395_test', img)


Obtain the threshold image for gray scale image and blue channel

In [312]:
def threshold_image (img):
    img = cv.GaussianBlur(img,(5,5),0)
    _,img = cv.threshold(img,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
    kernel = np.ones((10,10),np.uint8)
    img = cv.erode(img,kernel,iterations = 1) 
    return img

Obtain the threshold image to highlight the black corners

In [313]:
def highlight_black_corners (img):
    img = cv.GaussianBlur(img,(5,5),0)
    y, x = img.shape
    for i in range(5):
        img[i,:] = 0
        img[:,i] = 0
        img[:,x-i-1] = 0
        img[y-i-1,:] = 0
    _,img = cv.threshold(img,15,255,cv.THRESH_BINARY)
    return img

Remove black corners remaining from the lens of the camera used to take the lesion image 

In [314]:
def remove_corners(img):
    y, x = img.shape

    corners = {
        'top_left': {
            'x': [0,10],
            'y': [0,10]
        },
        'top_right': {
            'x': [0,10],
            'y': [y - 11, y-1]
        },
        'bottom_left': {
            'x': [x - 11, x-1],
            'y': [0,10]
        },
        'bottom_right': {
            'x': [x - 11, x-1],
            'y': [y - 11, y-1]
        },
    }

    for position in corners:
        for x in range(corners[position]['x'][0], corners[position]['x'][1]):
            for y in range(corners[position]['y'][0], corners[position]['y'][1]):
                if img[y,x] >= 255:
                    cv.floodFill(img, None, (x,y), 0)
    return img

Add gray and blue images

In [315]:
def add_gray_and_blue(img_1, img_2):
    img_sum = img_1 + img_2
    img_sum[img_sum > 125] = 255
    img_sum[img_sum < 125] = 0
    return img_sum
    

Remove minimal blobs from the image

In [316]:
def remove_minimal_blobs(img):

    kernel = np.ones((10,10),np.uint8)
    img_clean = cv.erode(img,kernel,iterations = 2)

    contours, _ = cv.findContours(img_clean, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    img_clean = np.zeros_like(img)
    for contour in contours:
        area = cv.contourArea(contour)
        if area > 8000:
            cv.fillPoly(img_clean, [contour], 255)


    return img_clean

Get coordinates of rectangle around ROI

In [317]:
def get_coordinates(img, gap):
    contours, _ = cv.findContours(img,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)

    if len(contours) == 0:
        return []

    y, x = img.shape
    coords = []
    
    t, b, r, l = [-1, 9999999, -1, 9999999]
    
    for points in contours:
        for p in points:
            if t < p[0][1]:
                t = p[0][1]
            if b > p[0][1]:
                b = p[0][1]
            if l > p[0][0]:
                l = p[0][0]
            if r < p[0][0]:
                r = p[0][0]

    coords.append(t + gap if t + gap < y else y)
    coords.append(b - gap if b - gap > -1 else 0)
    coords.append(r + gap if r + gap < x else x)
    coords.append(l - gap if l - gap > -1 else 0)

    return coords

Draw rectangle based on found coordinates

In [318]:
def draw_rectangle(img, coords):

    [t,b,r,l] = coords
                
    img_final = cv.rectangle(img, (l,b), (r,t), (255, 0, 0), 3)

    return img_final

Crop in coordinates     

In [319]:
def crop_image(img, coords):
    if len(coords) != 4:
        return img
    
    [t,b,r,l] = coords
    crop = img[b:t, l:r, :]

    return crop

Get maximized coordinates

In [320]:
def get_max_coords(max_coords, coords):
    if len(coords) != 4:
        return max_coords

    [t_m,b_m,r_m,l_m] = max_coords
    [t,b,r,l] = coords

    t_m = t if t_m > t else t_m
    b_m = b if b_m < b else b_m
    r_m = r if r_m < r else r_m
    l_m = l if l_m > l else l_m

    return [t_m,b_m,r_m,l_m]

In [321]:
max_coords = [-1, 9999999, -1, 9999999]
img_name = ''

data = {
    'name': [],
    'coords': []
}

try:
    for img_name in all_images:
        
        data['name'].append(img_name)
        img_rgb = read_image(img_name)
        img_rgb_2 = morphological_operations(img_rgb)
        img_black_corners = highlight_black_corners(cv.cvtColor(img_rgb_2, cv.COLOR_BGR2GRAY))
        img_black_corners_no_noise = remove_minimal_blobs(img_black_corners)
        coords_1 = get_coordinates(img_black_corners_no_noise, 0)
        crop = crop_image(img_rgb_2, coords_1)
        img_rgb = crop_image(img_rgb, coords_1)

        img_gray = cv.cvtColor(crop, cv.COLOR_BGR2GRAY)
        img_blue = crop[:,:,0]

        img_thresh_gray = threshold_image(img_gray)
        img_thresh_blue = threshold_image(img_blue)
        
        # save_in_test(img_name+'_thresh_gray', img_thresh_gray)
        # save_in_test(img_name+'_thresh_blue', img_thresh_blue)

        img_gray_no_corners = remove_corners(img_thresh_gray)
        img_blue_no_corners = remove_corners(img_thresh_blue)

        # save_in_test(img_name+'_gray_no_corners', img_gray_no_corners)
        # save_in_test(img_name+'_blue_no_corners', img_blue_no_corners)

        img_sum = add_gray_and_blue(img_gray_no_corners, img_blue_no_corners)
        img_clean = remove_minimal_blobs(img_sum)

        # save_in_test(img_name+'_clean', img_clean)

        coords = get_coordinates(img_clean, 100)
        
        img_final = crop_image(img_rgb, coords)
        # print(coords, img_final.shape)
        max_coords = get_max_coords(max_coords, coords)

        data['coords'].append(' '.join(map(str,coords if len(coords) > 0 else coords_1)))

        save_image(img_name+'_final', img_final)
except Exception as e: 
    print(img_name)
    print(e)

data['name'].append('max_coords')
data['coords'].append(', '.join(map(str,max_coords)))

In [ ]:
import pandas as pd

pd.DataFrame(data).drop_duplicates().to_csv('..\datasets\ISIC2017\Training_data_processed\\not_successfully_cutted\cut_coords.csv')